In [ ]:
%run ../ipynb_util_tars.py

In [1]:
import torch
from transformers import AutoTokenizer
from lxt.models.llama import LlamaForCausalLM, attnlrp
from lxt.utils import pdf_heatmap, clean_tokens

model = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# apply AttnLRP rules
attnlrp.register(model)

prompt = """\
Context: Mount Everest attracts many climbers, including highly experienced mountaineers. There are two main climbing routes, one approaching the summit from the southeast in Nepal (known as the standard route) and the other from the north in Tibet. While not posing substantial technical climbing challenges on the standard route, Everest presents dangers such as altitude sickness, weather, and wind, as well as hazards from avalanches and the Khumbu Icefall. As of November 2022, 310 people have died on Everest. Over 200 bodies remain on the mountain and have not been removed due to the dangerous conditions. The first recorded efforts to reach Everest's summit were made by British mountaineers. As Nepal did not allow foreigners to enter the country at the time, the British made several attempts on the north ridge route from the Tibetan side. After the first reconnaissance expedition by the British in 1921 reached 7,000 m (22,970 ft) on the North Col, the 1922 expedition pushed the north ridge route up to 8,320 m (27,300 ft), marking the first time a human had climbed above 8,000 m (26,247 ft). The 1924 expedition resulted in one of the greatest mysteries on Everest to this day: George Mallory and Andrew Irvine made a final summit attempt on 8 June but never returned, sparking debate as to whether they were the first to reach the top. Tenzing Norgay and Edmund Hillary made the first documented ascent of Everest in 1953, using the southeast ridge route. Norgay had reached 8,595 m (28,199 ft) the previous year as a member of the 1952 Swiss expedition. The Chinese mountaineering team of Wang Fuzhou, Gonpo, and Qu Yinhua made the first reported ascent of the peak from the north ridge on 25 May 1960. \
Question: How high did they climb in 1922? According to the text, the 1922 expedition reached 8,"""

input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

In [2]:
output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits

In [6]:
output_logits.shape

torch.Size([1, 504, 32000])

In [3]:
max_logits, max_indices = torch.max(output_logits[0, -1, :], dim=-1)

max_logits.backward(max_logits)
relevance = input_embeds.grad.float().sum(-1).cpu()[0]

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

In [4]:
relevance

tensor([ 1.3196e-01,  5.1883e-03,  3.0550e-03,  3.5725e-03,  3.4653e-03,
         4.6311e-03,  1.7928e-03,  1.2944e-03,  1.6060e-03,  2.7856e-03,
         2.4524e-03, -1.2477e-03,  1.6194e-03,  1.7904e-03,  1.7456e-03,
         1.8251e-03,  2.4546e-03,  2.1118e-03,  1.2877e-03,  1.8194e-03,
         1.8732e-03,  1.9507e-03,  1.3257e-03,  2.8442e-03,  1.9830e-03,
         2.3725e-03,  1.3156e-03,  3.8082e-04,  2.0710e-03,  1.3968e-03,
         2.2734e-03,  1.1085e-03,  1.6843e-03,  1.2335e-03,  1.6933e-03,
         1.6841e-03,  2.0773e-03,  1.9222e-03,  1.8050e-03,  1.4322e-03,
         5.8241e-04,  1.0884e-03,  9.0586e-04,  1.1562e-03,  9.9855e-04,
         1.3892e-03,  1.3004e-03,  1.0053e-03,  4.3004e-04,  1.0687e-03,
         9.2276e-04,  1.2031e-03,  1.7044e-03,  1.7856e-03,  1.4967e-03,
         2.8682e-03,  8.0016e-04,  1.1782e-03,  1.0711e-03,  1.2500e-03,
         1.2540e-03,  1.7711e-03,  2.7716e-03,  1.9745e-03,  1.4472e-03,
         1.2429e-03,  1.3865e-03,  9.8715e-04,  1.1

In [5]:
pdf_heatmap(tokens, relevance, path='heatmap.pdf', backend='xelatex')

FileNotFoundError: [Errno 2] No such file or directory: 'xelatex'